In [ ]:
import os
import requests
from datetime import datetime, timedelta
from langchain.llms import OpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

os.environ["OPENWEATHERMAP_API_KEY"] = "{{OPENWEATHERMAP_API_KEY}}"

# Insert your key here
llm = OpenAI(temperature=0.0,
            openai_api_key = "{{OpenAI_Key}}")

# Function to get current weather data
def get_weather_data(city):
    api_key = os.environ["OPENWEATHERMAP_API_KEY"]
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# Function to get weather forecast data
def get_weather_forecast(city):
    api_key = os.environ["OPENWEATHERMAP_API_KEY"]
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# loading tools
tools = load_tools(["llm-math"], llm=llm)

agent = initialize_agent(tools, 
                        llm, 
                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                        verbose=True)

# user's query
city_name = "Dallas"
current_weather = get_weather_data(city_name)
weather_forecast = get_weather_forecast(city_name)

if "main" in current_weather:
    current_temp = current_weather['main']['temp']
    print(f"The current temperature in {city_name} is {current_temp}°C.")
    
    forecast_temps = [forecast['main']['temp'] for forecast in weather_forecast['list']]
    average_forecast_temp = sum(forecast_temps) / len(forecast_temps)
    print(f"The average temperature in {city_name} for the next few days is {average_forecast_temp:.2f}°C.")
else:
    print(f"Could not retrieve the weather data for {city_name}. Please check the city name and try again.")
